In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

In [4]:
# Define parameters for simulation
P_MAX = 100
P_MIN = 20
K = 5 # number of areas for creating price ranges

In [ ]:
# First focus on single product case
# Demand is generated from a poisson process with a mean equal to that
# predicted by a linear demand curve

In [5]:
theta_max = 1-P_MIN/P_MAX # Maximum discount allowed


0.8

In [ ]:
# Load and view data, total 7 data tables
# 1. SKUs Data
skus = pd.read_csv('sku_data.csv')
# 2. Users data
users = pd.read_csv('user_data.csv')
# 3. Order data
orders = pd.read_csv('order_data.csv')
# 4. Click data
clicks = pd.read_csv('click_data.csv')
# 5. Delivery data
delivery = pd.read_csv('delivery_data.csv')
# 6. Inventory data
inventory = pd.read_csv('inventory_data.csv')
# 7. Network data
network = pd.read_csv('network_data.csv')

In [ ]:
# Sample of skus data
print("size = {}\nShape = {}".format(skus.size, skus.shape))
print(skus.columns)

In [ ]:
skus.head()

In [ ]:
skus['brand_ID'].unique().size

In [ ]:
 # Sample of users data
users.head()

In [ ]:
 # Sample of clicks data
clicks.head()

In [ ]:
 # Sample of orders data
orders.head().T

In [ ]:
orders['sku_ID'].unique().size

In [ ]:
# Sample of delivery data
delivery.head()

In [ ]:
# Sample of inventory data
inventory.head()

In [ ]:
# Sample of network data
network.head()

In [ ]:
# Let's explore the data further by looking at the full customer experience of a random customer
orders[orders['order_ID']=='81a6fa818d'].T

In [ ]:
# Let's look at the customer with user_ID = '2c511cbd9e' from users table
users[users['user_ID'] == '2c511cbd9e']

In [ ]:
# Next let's check sku information from skus table
skus[skus['sku_ID'].isin(['ac61f4e10e', 'eb3f2d2fd8'])]

In [ ]:
# clicks table provides further information on the roadmap leading to this purchase
clicks[clicks['sku_ID']=='eb3f2d2fd8']

In [ ]:
clicks['request_date'] = clicks['request_time'].str.split(expand=True)[0]

In [ ]:
clicks.head()

In [ ]:
# clicks table provides further information on the roadmap leading to this purchase
clicks[(clicks['sku_ID']=='eb3f2d2fd8') & (clicks['request_date']=='2018-03-01')]

In [ ]:
# Next we look at how the order is fullfilled for this curtomer
orders[orders['order_ID']=='81a6fa818d'][['sku_ID', 'dc_ori', 'dc_des']]

In [ ]:
# delivery table provides additional information
delivery[delivery['order_ID']=='81a6fa818d']

In [ ]:
# inventory table provides more insights on the fulfillment logic
inventory[(inventory['sku_ID'].isin(['ac61f4e10e', 'eb3f2d2fd8'])) & \
         (inventory['date']=='2018-03-01') & (inventory['dc_ID']==9)]

In [ ]:
# inventory table provides more insights on the fulfillment logic
inventory[(inventory['sku_ID'].isin(['ac61f4e10e', 'eb3f2d2fd8'])) & \
         (inventory['date']=='2018-03-01') & (inventory['dc_ID']==27)]

In [ ]:
orders[(orders['sku_ID']=='ac61f4e10e') & (orders['order_date']=='2018-03-01')]

In [ ]:
orders.shape

In [ ]:
# Create new date frame with relevant features of an order
orders1 = orders[orders['original_unit_price']!=0][['sku_ID','order_date','quantity',
                  'original_unit_price','final_unit_price',
                  'direct_discount_per_unit','quantity_discount_per_unit',
                 'bundle_discount_per_unit', 'coupon_discount_per_unit']]
orders1.head()

In [ ]:
# Aggregrate sales by sku_ID and order_date
orders2 = orders1.groupby(['order_date', 'sku_ID'], as_index=False).agg('sum').rename(columns={
    'quantity':'total_quantity',
    'original_unit_price':'original_total_price',
    'final_unit_price':'final_total_price',
    'direct_discount_per_unit':'direct_discount_total',
    'quantity_discount_per_unit':'quantity_discount_total',
    'bundle_discount_per_unit':'bundle_discount_total',
    'coupon_discount_per_unit':'coupon_discount_total'
})
orders2.shape

In [ ]:
orders2.head(10)

In [ ]:
orders2['avg_original_unit_price'] = orders2['original_total_price']/orders2['total_quantity']
orders2['avg_final_unit_price'] = orders2['final_total_price']/orders2['total_quantity']

In [ ]:
orders2.head(10)

In [ ]:
sku_01326007f9 = orders2[orders2['sku_ID']=='01326007f9'][['total_quantity','avg_final_unit_price']]

In [ ]:
plt.plot(sku_01326007f9['avg_final_unit_price'], sku_01326007f9['total_quantity'],'ro')
plt.show()